In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data # 获取迭代数据
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.autograd import Variable # 获取变量
import torch.optim as optim


# import torchvision
# from torchvision.datasets import mnist # 获取数据集
# import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import cooler

## 准备数据集

In [4]:
#K562的一个单细胞
pair_path = "raw_data/K562/GSM2109985_90_K562_SetB-40kb.csv.gz"
df = pd.read_csv(pair_path,compression="gzip")



/tmp/ipykernel_848/2570173081.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(pair_path,compression="gzip")


In [5]:
df.head()

,chr1,chr2,start1,end1,start2,end2,count
0,1,1,760000,800000,156040000,156080000,1
1,1,1,800000,840000,800000,840000,1
2,1,1,800000,840000,1000000,1040000,1
3,1,1,880000,920000,920000,960000,1
4,1,1,1080000,1120000,1080000,1120000,1


In [20]:
df['pos1'] = (df['start1'] + df['end1'] )// 2
df['pos2'] = (df['start2'] + df['end2'] )// 2
df = df[['chr1','pos1','chr2','pos2','count']]
#chr1列所有元素前面加上"chr"
df['chr1'] = df['chr1'].apply(lambda x: "chr" + str(x))
df['chr2'] = df['chr2'].apply(lambda x: "chr" + str(x))

In [5]:
df.to_csv("../data/GSM2109985_90_K562_SetB-40kb.pairs.gz",sep="\t",compression="gzip",index=None,header=None)

## 处理cool文件

In [3]:
# cool_path='GSM2109985_90_K562_SetB-40kb.cool'
# c = cooler.Cooler(cool_path)

#通过cooler读入GSM2109985_90_K562_SetB-40kb.mcool
c = cooler.Cooler("./data/GSM2109985_90_K562_SetB.mcool::/resolutions/400000")

In [4]:
#只选择
c.pixels(join=True)[100:]

,chrom1,start1,end1,chrom2,start2,end2,count
100,chr1,3200000,3600000,chr4,90000000,90400000,1
101,chr1,3200000,3600000,chr4,184400000,184800000,1
102,chr1,3200000,3600000,chr4,185600000,186000000,1
103,chr1,3200000,3600000,chr4,186000000,186400000,1
104,chr1,3200000,3600000,chrX,120400000,120800000,1
...,...,...,...,...,...,...,...
95312,chr21,46800000,47200000,chr21,46800000,47200000,2
95313,chr21,46800000,47200000,chr21,47200000,47600000,1
95314,chr21,47200000,47600000,chr21,47200000,47600000,6
95315,chr21,47600000,48000000,chr21,47600000,48000000,10


In [5]:
c.bins()[:]

,chrom,start,end
0,chr1,0,400000
1,chr1,400000,800000
2,chr1,800000,1200000
3,chr1,1200000,1600000
4,chr1,1600000,2000000
...,...,...,...
7745,chr21,46400000,46800000
7746,chr21,46800000,47200000
7747,chr21,47200000,47600000
7748,chr21,47600000,48000000


In [7]:
matrix1 = c.matrix(balance=False)

In [14]:
bin_table = c.bins()[:]

In [44]:
window = 21
x1 = 4940
y1 = 4940
x2 = x1 + window
y2 = y1 + window
submatrix1 = matrix1[x1:x2, y1:y2]
#统计submatrix1中有多少0
nozeron = np.count_nonzero(submatrix1)

tmp_bin_x = bin_table.loc[x1:x2,:]
tmp_bin_y = bin_table.loc[y1:y2,:]


In [42]:
bin_table[(bin_table['chrom']=="chr11") &(bin_table['start'] < 5312156 ) & (5312156 < bin_table['end'] )]

,chrom,start,end
4946,chr11,5200000,5600000


In [43]:
bin_table[(bin_table['chrom']=="chr11") &(bin_table['start'] < 5443877 ) & (5443877 < bin_table['end'] )]

,chrom,start,end
4946,chr11,5200000,5600000


### label

In [2]:
sv_list = pd.read_csv("./data/sv_list.csv")
k562_list = sv_list[sv_list['Cell Line']=="K562"]

In [50]:
def determine_label(tmp_bin_x,tmp_bin_y,k562_list):
    label = "no"
    for i in range(len(tmp_bin_x)):
        chr1 = tmp_bin_x['chrom'].iat[i]
        start1 = tmp_bin_x['start'].iat[i]
        end1 = tmp_bin_x['end'].iat[i]
        for j in range(len(tmp_bin_y)):
            chr2 = tmp_bin_y['chrom'].iat[j]
            start2 = tmp_bin_y['start'].iat[j]
            end2 = tmp_bin_y['end'].iat[j]

            for m in range(len(k562_list)):
                chrom1 = k562_list['chrom1'].iat[m]
                chrom2 = k562_list['chrom2'].iat[m]
                breakpoint1 = k562_list['breakpoint1'].iat[m]
                breakpoint2 = k562_list['breakpoint2'].iat[m]
                string = k562_list['strands'].iat[m]
                if (chr1 == chrom1 and chr2==chrom2 and (breakpoint1 > start1 and breakpoint1<end1) and (breakpoint2 > start2 and breakpoint2<end2) ):
                    label = string
                    break
                elif (chr1 == chrom2 and chr2==chrom1 and (breakpoint1 > start2 and breakpoint1<end2) and (breakpoint2 > start1 and breakpoint2<end1)):
                    label = string
                    break
            if label != "no":
                break
        if label != "no":
            break
    return label

          

In [48]:
label

'+-'

In [23]:
#矩阵数据
#生成100*1*21*21的随机tensor
x = torch.randn(100,1,21,21)


In [135]:
#找到有label对应的位置
pos_data_list = []
pos_label_list = []

for i in range(len(k562_list)):
    chrom1 = k562_list['chrom1'].iat[i]
    chrom2 = k562_list['chrom2'].iat[i]
    breakpoint1 = k562_list['breakpoint1'].iat[i]
    breakpoint2 = k562_list['breakpoint2'].iat[i]
    string = k562_list['strands'].iat[i]

    #找到对应的bin
    bin1 = bin_table[(bin_table['chrom']==chrom1) &(bin_table['start'] < breakpoint1 ) & (breakpoint1 < bin_table['end'] )]
    bin2 = bin_table[(bin_table['chrom']==chrom2) &(bin_table['start'] < breakpoint2 ) & (breakpoint2 < bin_table['end'] )]

    #找到对应的matrix
    x1 = bin1.index[0]
    y1 = bin2.index[0]

    x2 = x1 + window
    y2 = y1 + window
    submatrix = matrix1[x1:x2, y1:y2]

    #pos_data_list中添加submatrix
    pos_data_list.append(submatrix)
    pos_label_list.append(string)



In [126]:
#很少，但是先这样吧
neg_data_list = []
neg_label_list = []


sample_x1=np.random.randint(7000, size=(50))
sample_y1=np.random.randint(7000, size=(50))
sample_x2 = sample_x1 + window
sample_y2 = sample_y1 + window

for x,y in zip(sample_x1,sample_y1):
    submatrix = matrix1[x:x+window, y:y+window]
    nozeron = np.count_nonzero(submatrix)
    tmp_bin_x = bin_table.loc[x:x+window,:]
    tmp_bin_y = bin_table.loc[y:y+window,:]
    #得到标签
    label = determine_label(tmp_bin_x,tmp_bin_y,k562_list)
    if(nozeron>5):
        neg_data_list.append(submatrix)
        neg_label_list.append(label)


In [136]:
all_data = pos_data_list + neg_data_list
all_label = pos_label_list + neg_label_list

In [13]:
# 保存
# a=np.array(all_data)
np.save('data/data.npy',a)   # 保存为.npy格式
# b = np.array(all_label)

# 读取


In [3]:
a = np.load('data/data.npy')

In [16]:
a = a.reshape(47,1,21,21)

In [12]:
#a转成float
a = a.astype(np.float32)

In [189]:
np.unique(b)

array(['++', '+-', '-+', '--', 'no'], dtype='<U2')

In [191]:
label_dic = {'++':0,'+-':1,'-+':2,'--':3,'no':4}

In [192]:
#将b按照label_dic转成对应int
b = [label_dic[i] for i in b]

In [194]:
np.save('data/label.npy',b)

In [149]:
a_tensor = torch.tensor(a,dtype=float)

In [153]:
a_tensor = a_tensor.reshape(47,1,21,21)

In [160]:
#save a_tensor

torch.save(a_tensor, "data/data.pt")





In [161]:
y = torch.load("data/data.pt")

In [163]:
y.shape

torch.Size([47, 1, 21, 21])

In [201]:
class CustomDataset_attr(Dataset):
    """
    自定义数据集，用于读取celebA数据集中的属性识别（attribute recognition）数据的标签和图像
    图像格式为jpg
    """
    
    def __init__(self,label_file, data_file, transform = None):
    
        super().__init__()
        self.attr_ = np.load(label_file).tolist()
        self.label_file = label_file
        self.data_file = data_file
        self.transform = transform
    
    def __len__(self):
        #展示数据中总共有多少个样本
        print("hi")
        return len(self.attr_)
    
    def __info__(self):
        print("CustomData")
        print("\t Number of samples: {}".format(len(self.attr_)-1))

    
    def __getitem__(self,idx):
        #保证idx不是一个tensor
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        self.attr_ = np.load(self.label_file).tolist()
        #加载tensor
        image = torch.load(self.data_file)[idx]
        label = int(self.attr_[idx])
        
        if self.transform != None:
            image = self.transform(image)
            
        sample = (image,label)
        return sample

In [200]:

label_file = 'data/label.npy'
data_file = 'data/data.pt' #tensor

data = CustomDataset_attr(label_file,data_file,transform=None)